In [1]:
import time
from datetime import datetime
from database.select.db_select import get_entire_season
from data.data import Game, Team, TeamGame

In [2]:
season_info, teams, games = await get_entire_season(2023)

In [ ]:
def game_analysis(teams: list[Team], games: list[Game], regular_season_games: int) -> tuple[dict[str, Team], dict[str, list[TeamGame]]]:
    team_dict: dict[str, Team] = {team.full_name: team for team in teams}
    team_game_dict: dict[str, list[TeamGame]] = {}
    
    for game in games:
        away_team: str = game.away_team
        home_team: str = game.home_team
        division_game: bool = team_dict[away_team].division == team_dict[home_team].division
        conference_game: bool = team_dict[away_team].conference == team_dict[home_team].conference
        playoff_game: bool = game.week > regular_season_games
        
        if away_team not in team_game_dict:
            team_game_dict[away_team] = []
        if home_team not in team_game_dict:
            team_game_dict[home_team] = []
            
        team_dict[away_team].points_for += game.away_score
        team_dict[away_team].points_against += game.home_score
        team_dict[away_team].point_differential += game.away_score - game.home_score
        team_dict[home_team].points_for += game.home_score
        team_dict[home_team].points_against += game.away_score
        team_dict[home_team].point_differential += game.home_score - game.away_score
            
        if game.away_score > game.home_score:
            away_result: chr = 'W'
            home_result: chr = 'L'
            team_dict[away_team].wins += 1
            team_dict[home_team].losses += 1
            if conference_game:
                team_dict[away_team].conference_wins += 1
                team_dict[home_team].conference_losses += 1
                if division_game:
                    team_dict[away_team].division_wins += 1
                    team_dict[home_team].division_losses += 1
        elif game.away_score < game.home_score:
            away_result: chr = 'L'
            home_result: chr = 'W'
            team_dict[away_team].losses += 1
            team_dict[home_team].wins += 1
            if conference_game:
                team_dict[away_team].conference_losses += 1
                team_dict[home_team].conference_wins += 1
                if division_game:
                    team_dict[away_team].division_losses += 1
                    team_dict[home_team].division_wins += 1
        else:
            away_result: chr = 'T'
            home_result: chr = 'T'
            team_dict[away_team].ties += 1
            team_dict[home_team].ties += 1
            if conference_game:
                team_dict[away_team].conference_ties += 1
                team_dict[home_team].conference_ties += 1
                if division_game:
                    team_dict[away_team].division_ties += 1
                    team_dict[home_team].division_ties += 1
            
        team_game_dict[away_team].append(
            TeamGame(
                game.week,
                game.week_name,
                home_team,
                game.away_score,
                game.home_score,
                away_result,
                home_result,
                False,
                division_game,
                conference_game,
                playoff_game)
        )
        
        team_game_dict[home_team].append(
            TeamGame(
                game.week,
                game.week_name,
                away_team,
                game.home_score,
                game.away_score,
                away_result,
                home_result,
                True,
                division_game,
                conference_game,
                playoff_game
            )
        )
        

In [28]:


def other_game_info(row: pd.Series, regular_season_games: int) -> pd.Series:
    if row['AwayTeam'] == row['Team']:
        team_score = row['AwayScore']
        opponent_score = row['HomeScore']
    else:
        team_score = row['HomeScore']
        opponent_score = row['AwayScore']
        
    if team_score > opponent_score:
        result: chr = 'W'
    elif team_score < opponent_score:
        result: chr = 'L'
    else:
        result: chr = 'T'
    
    row['Opponent'] = row['HomeTeam'] if row['Team'] == row['AwayTeam'] else row['AwayTeam']
    row['TeamScore'] = team_score
    row['OpponentScore'] = opponent_score
    row['Result'] = result
    row['HomeGame'] = row['HomeTeam'] == row['Team']
    row['PlayoffGame'] = row['Week'] > regular_season_games
    
    return row

def team_games(games_df: pd.DataFrame, team: str, regular_season_games: int) -> pd.DataFrame:
    teams_games_df = games_df[((games_df['HomeTeam'] == team) | (games_df['AwayTeam'] == team))]
    teams_games_df.insert(0, 'Team', team)
    teams_games_df = teams_games_df.apply(other_game_info, regular_season_games=regular_season_games, axis=1)
    
    return teams_games_df
    
team_game_dfs: dict[str, pd.DataFrame] = {team: team_games(games_df, team, season_info.regular_season_week_count) for team in teams_df['FullName'].to_list()}

In [29]:
def result_counts(column: pd.Series) -> pd.Series:
    results: dict[chr, int] = column.value_counts().to_dict()
    
    wins: int = results['W'] if 'W' in results else 0
    losses: int = results['L'] if 'L' in results else 0
    ties: int = results['T'] if 'T' in results else 0
    
    return wins, losses, ties

def win_percentage(wins: int, ties: int, num_games: int) -> float:
    if num_games == 0:
        return 0.0
    
    return round((wins + (ties / 2.0)) / num_games, 3)

def record(row: pd.Series, team_game_dfs: dict[str, pd.DataFrame], teams_df: pd.DataFrame) -> pd.Series:
    
    start = time.perf_counter()
    team_game_df = team_game_dfs[row['FullName']]
    team_game_df = team_game_df[~team_game_df['PlayoffGame']]
    
    wins, losses, ties = result_counts(team_game_df['Result'])
    
    row['W'] = wins
    row['L'] = losses
    row['T'] = ties
    row['Pct'] = win_percentage(wins, ties, len(team_game_df))
    
    conference_teams: list[str] = teams_df[teams_df['Conference'] == row['Conference']]['FullName'].to_list()
    conference_games_df: pd.DataFrame = team_game_df[(team_game_df['Opponent'].isin(conference_teams))].copy()
    
    wins, losses, ties = result_counts(conference_games_df['Result'])
    
    row['Conf'] = f'{wins}-{losses}-{ties}'
    row['PctConf'] = win_percentage(wins, ties, len(conference_games_df))
    
    division_teams: list[str] = teams_df[(teams_df['Division'] == row['Division'])]['FullName'].to_list()
    division_games_df: pd.DataFrame = conference_games_df[(conference_games_df['Opponent'].isin(division_teams))].copy()
    
    wins, losses, ties = result_counts(division_games_df['Result'])
    
    row['Div'] = f'{wins}-{losses}-{ties}'
    row['PctDiv'] = win_percentage(wins, ties, len(division_games_df))
    
    home_games_df: pd.DataFrame = team_game_df[team_game_df['HomeGame']].copy()
    wins, losses, ties = result_counts(home_games_df['Result'])
    row['Home'] = f'{wins}-{losses}-{ties}'
    
    away_games_df: pd.DataFrame = team_game_df[~team_game_df['HomeGame']].copy()
    wins, losses, ties = result_counts(away_games_df['Result'])
    row['Away'] = f'{wins}-{losses}-{ties}'
    
    
    end = time.perf_counter()
    print(f"{row['FullName']} record created in {end - start:0.4f} seconds")
    
    return row

teams_df = teams_df.apply(record, team_game_dfs=team_game_dfs, teams_df=teams_df, axis=1)

teams_df.head(32)

Buffalo Bills record created in 0.0048 seconds
Miami Dolphins record created in 0.0044 seconds
New York Jets record created in 0.0036 seconds
New England Patriots record created in 0.0034 seconds
Kansas City Chiefs record created in 0.0042 seconds
Las Vegas Raiders record created in 0.0032 seconds
Denver Broncos record created in 0.0029 seconds
Los Angeles Chargers record created in 0.0030 seconds
Baltimore Ravens record created in 0.0036 seconds
Cleveland Browns record created in 0.0037 seconds
Pittsburgh Steelers record created in 0.0038 seconds
Cincinnati Bengals record created in 0.0030 seconds
Houston Texans record created in 0.0036 seconds
Jacksonville Jaguars record created in 0.0031 seconds
Indianapolis Colts record created in 0.0029 seconds
Tennessee Titans record created in 0.0029 seconds
Dallas Cowboys record created in 0.0035 seconds
Philadelphia Eagles record created in 0.0035 seconds
New York Giants record created in 0.0029 seconds
Washington Commanders record created in 

,Location,Name,FullName,Division,Conference,W,L,T,Pct,Conf,PctConf,Div,PctDiv,Home,Away
0,Buffalo,Bills,Buffalo Bills,AFC East,AFC,11,6,0,0.647,7-5-0,0.583,4-2-0,0.667,7-2-0,4-4-0
1,Miami,Dolphins,Miami Dolphins,AFC East,AFC,11,6,0,0.647,7-5-0,0.583,4-2-0,0.667,7-2-0,4-4-0
2,New York,Jets,New York Jets,AFC East,AFC,7,10,0,0.412,4-8-0,0.333,2-4-0,0.333,4-5-0,3-5-0
3,New England,Patriots,New England Patriots,AFC East,AFC,4,13,0,0.235,4-8-0,0.333,2-4-0,0.333,1-8-0,3-5-0
4,Kansas City,Chiefs,Kansas City Chiefs,AFC West,AFC,11,6,0,0.647,9-3-0,0.750,4-2-0,0.667,5-4-0,6-2-0
5,Las Vegas,Raiders,Las Vegas Raiders,AFC West,AFC,8,9,0,0.471,6-6-0,0.500,4-2-0,0.667,6-3-0,2-6-0
6,Denver,Broncos,Denver Broncos,AFC West,AFC,8,9,0,0.471,5-7-0,0.417,3-3-0,0.500,5-4-0,3-5-0
7,Los Angeles,Chargers,Los Angeles Chargers,AFC West,AFC,5,12,0,0.294,3-9-0,0.250,1-5-0,0.167,2-7-0,3-5-0
8,Baltimore,Ravens,Baltimore Ravens,AFC North,AFC,13,4,0,0.765,8-4-0,0.667,3-3-0,0.500,6-3-0,7-1-0
9,Cleveland,Browns,Cleveland Browns,AFC North,AFC,11,6,0,0.647,8-4-0,0.667,3-3-0,0.500,8-1-0,3-5-0


In [30]:
for division in teams_df['Division'].unique():
    teams_df[teams_df['Division'] == division].sort_values(by='Pct', ascending=False)